<a href="https://colab.research.google.com/github/cateto/python4NLP/blob/main/cohesion_test/%5Bparaphrase_multilingual_mpnet_base_v2%5Dsentence_transformer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# sentence transformers 로딩

In [14]:
pip install -U sentence-transformers

In [15]:
import math
import logging
from datetime import datetime

import torch
from torch.utils.data import DataLoader
from sentence_transformers import SentenceTransformer,  LoggingHandler, losses, models, util
from sentence_transformers.evaluation import EmbeddingSimilarityEvaluator
from sentence_transformers.readers import InputExample

In [16]:
import torch
from sentence_transformers import SentenceTransformer, util

model_name = 'sentence-transformers/paraphrase-multilingual-mpnet-base-v2'


embedding_model = models.Transformer(model_name)

pooler = models.Pooling(
    embedding_model.get_word_embedding_dimension(),
    pooling_mode_mean_tokens=True,
    pooling_mode_cls_token=False,
    pooling_mode_max_tokens=False,
)

model = SentenceTransformer(modules=[embedding_model, pooler])


Downloading:   0%|          | 0.00/723 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.04G [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/402 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/4.83M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/8.66M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/239 [00:00<?, ?B/s]

# 명사,동사,형용사 필터링

In [17]:
# Colab에 Mecab 설치
!git clone https://github.com/SOMJANG/Mecab-ko-for-Google-Colab.git
%cd Mecab-ko-for-Google-Colab
!bash install_mecab-ko_on_colab190912.sh

Cloning into 'Mecab-ko-for-Google-Colab'...
remote: Enumerating objects: 91, done.
remote: Counting objects: 100% (91/91), done.
remote: Compressing objects: 100% (85/85), done.
remote: Total 91 (delta 43), reused 22 (delta 6), pack-reused 0
Unpacking objects: 100% (91/91), done.
/content/Mecab-ko-for-Google-Colab/Mecab-ko-for-Google-Colab
Installing konlpy.....
Done
Installing mecab-0.996-ko-0.9.2.tar.gz.....
from https://bitbucket.org/eunjeon/mecab-ko/downloads/mecab-0.996-ko-0.9.2.tar.gz
--2021-11-01 00:26:38--  https://bitbucket.org/eunjeon/mecab-ko/downloads/mecab-0.996-ko-0.9.2.tar.gz
Resolving bitbucket.org (bitbucket.org)... 104.192.141.1, 2406:da00:ff00::34cc:ea4a, 2406:da00:ff00::22e9:9f55, ...
Connecting to bitbucket.org (bitbucket.org)|104.192.141.1|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://bbuseruploads.s3.amazonaws.com/eunjeon/mecab-ko/downloads/mecab-0.996-ko-0.9.2.tar.gz?Signature=o8Y3mBv5UsryJ1KohkflJjTq3dI%3D&Expires=163572

In [18]:
def cleaning(sentence):
  clean_words = []
  for word in okt.pos(sentence, stem=True):
    if word[1] in ['Noun', 'Verb', 'Adjective']: #명사, 동사, 형용사
      clean_words.append(word[0])
  print(clean_words)
  temp_sentence = ' '.join(clean_words)
  return temp_sentence, clean_words

In [19]:
from konlpy.tag import *
okt = Okt()

docs = [
    "1992년 7월 8일 손흥민은 강원도 춘천시 후평동에서 아버지 손웅정과 어머니 길은자의 차남으로 태어나 그곳에서 자랐다.",
    "형은 손흥윤이다.",
    "춘천 부안초등학교를 졸업했고, 춘천 후평중학교에 입학한 후 2학년때 원주 육민관중학교 축구부에 들어가기 위해 전학하여 졸업하였으며, 2008년 당시 FC 서울의 U-18팀이었던 동북고등학교 축구부에서 선수 활동 중 대한축구협회 우수선수 해외유학 프로젝트에 선발되어 2008년 8월 독일 분데스리가의 함부르크 유소년팀에 입단하였다.",
    "함부르크 유스팀 주전 공격수로 2008년 6월 네덜란드에서 열린 4개국 경기에서 4게임에 출전, 3골을 터뜨렸다.",
    "1년간의 유학 후 2009년 8월 한국으로 돌아온 후 10월에 개막한 FIFA U-17 월드컵에 출전하여 3골을 터트리며 한국을 8강으로 이끌었다.",
    "그해 11월 함부르크의 정식 유소년팀 선수 계약을 체결하였으며 독일 U-19 리그 4경기 2골을 넣고 2군 리그에 출전을 시작했다.",
    "독일 U-19 리그에서 손흥민은 11경기 6골, 2부 리그에서는 6경기 1골을 넣으며 재능을 인정받아 2010년 6월 17세의 나이로 함부르크의 1군 팀 훈련에 참가, 프리시즌 활약으로 함부르크와 정식 계약을 한 후 10월 18세에 함부르크 1군 소속으로 독일 분데스리가에 데뷔하였다.",
]

result_arr = []
result = []
for sentence in docs:
  temp_sentence, clean_words = cleaning(sentence)
  result_arr.append(clean_words)
  result.append(temp_sentence)

['손흥민', '강원도', '춘천시', '후평동', '아버지', '손웅정', '어머니', '길다', '자의', '차남', '태어나다', '곳', '자르다']
['형', '손흥윤']
['춘천', '부안', '초등학교', '졸업', '하다', '춘천', '후평', '중학교', '입학', '후', '학년', '때', '원주', '민', '관중', '학교', '축구', '부', '들어가다', '위해', '전학', '하다', '졸업', '하다', '당시', '서울', '팀', '이다', '동북', '고등학교', '축구', '부', '선수', '활동', '중', '축구', '협회', '우수', '선수', '해외', '유학', '프로젝트', '선발', '되어다', '독일', '분데스리가', '함부르크', '유', '소년', '팀', '입단', '하다']
['함부르크', '유', '스팀', '주전', '공격수', '네덜란드', '열리다', '개국', '경기', '게임', '출전', '골', '터뜨리다']
['유학', '후', '한국', '돌아오다', '후', '개막', '월드컵', '출전', '하다', '골', '터', '트리', '한국', '강', '이끌다']
['함부르크', '정식', '유', '소년', '팀', '선수', '계약', '체결', '하다', '독일', '리그', '경기', '골', '넣다', '군', '리그', '출전', '시작', '하다']
['독일', '리그', '손흥민', '경기', '골', '부', '리그', '경기', '골', '넣다', '재능', '인정받다', '세', '나이', '함부르크', '군', '팀', '훈련', '참가', '프리', '시즌', '활약', '함부르크', '정식', '계약', '하다', '후', '세', '함부르크', '군', '소속', '독일', '분데스리가', '데뷔', '하다']


In [20]:

document_embeddings = model.encode(docs)

query = "손흥민은 어린 나이에 유럽에 진출하였다."
temp_sentence, _ = cleaning(query)
query_embedding = model.encode(temp_sentence)
print(temp_sentence)
print(query_embedding.shape)
print(query_embedding)


top_k = min(5, len(docs))
cos_scores = util.pytorch_cos_sim(query_embedding, document_embeddings)[0]
top_results = torch.topk(cos_scores, k=top_k)

print(f"입력 문장: {query}")
print(f"<입력 문장과 유사한 {top_k} 개의 문장>")

for i, (score, idx) in enumerate(zip(top_results[0], top_results[1])):
    print(f"{i+1}: {docs[idx]} {'(유사도: {:.4f})'.format(score)}")

['손흥민', '어리다', '나이', '유럽', '진출', '하다']
손흥민 어리다 나이 유럽 진출 하다
(768,)
[-5.95532730e-02  1.74716339e-01 -1.81865450e-02 -2.75784638e-02
  5.06996103e-02 -8.99189040e-02  1.94792852e-01 -8.75617564e-02
  1.90199628e-01  4.90266345e-02  8.77372772e-02 -1.08249873e-01
 -1.10956207e-01 -2.37110388e-02  3.98612618e-02 -4.26584110e-02
  1.26201048e-01 -1.58845317e-02 -7.17853084e-02 -3.27889472e-02
  1.67470947e-02 -2.53705382e-02  1.66286267e-02  1.17162265e-01
 -1.81132667e-02  4.07319814e-02 -4.60506864e-02  5.94031774e-02
  9.38522369e-02  1.94523167e-02 -7.11729343e-04 -2.73072310e-02
  1.26573578e-01  1.33281097e-01  4.00558524e-02  8.55925977e-02
  3.18228528e-02  1.22636072e-02  1.60758719e-02 -1.58494398e-01
  1.89723179e-01  2.20579039e-02 -2.77483687e-02  4.59454916e-02
 -1.91990510e-02 -5.87718226e-02 -2.31559966e-02 -5.33058681e-02
 -1.97959840e-02  4.68929932e-02  6.47577550e-03  2.03001965e-02
 -2.54495870e-02  1.54982042e-02  7.30625466e-02 -4.87337075e-02
 -1.21783083e-02 -2.1692

In [21]:
sentence1='최근 국민들의 여행심리 위축 등으로 동남아 등 다른 노선까지 영향을 받는 상황이다.'
sentence2='동남아시아와 같은 다른 노선은 최근 사람들의 여행 감정의 하락에 영향을 받았습니다.'
sentence_emb1, _ = cleaning(sentence1)
sentence_emb1 = model.encode(sentence_emb1)
sentence_emb2, _ = cleaning(sentence2)
sentence_emb2 = model.encode(sentence_emb2)
cos_scores = util.pytorch_cos_sim(sentence_emb1, sentence_emb2)[0]
print(f" 입력 문장의 코사인 유사도 : {str(cos_scores.numpy()[0])} ")

['최근', '국민', '여행', '심리', '위축', '등', '동남아', '등', '다른', '노선', '영향', '받다', '상황']
['동남아시아', '같다', '다른', '노선', '최근', '사람', '여행', '감정', '하락', '영향', '받다']
 입력 문장의 코사인 유사도 : 0.93096846 
